In [2]:
import csv
import base64
import time
import fnmatch
import urllib
import traceback
from datetime import datetime, timedelta
from github import Github, StatsContributor
from awesomecure import awesome2py
from urllib.parse import urlparse
from os import getenv, path
from dotenv import load_dotenv
from termcolor import colored

import requests
from bs4 import BeautifulSoup

In [3]:
def crawl_dependents(repo,page_num):
    url = 'https://github.com/{}/network/dependents'.format(repo)
    dependents_data = []
    list_end = False
    
    for i in range(page_num):
        #print("GET " + url)
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")

        page_data = [
            "{}/{}".format(
                t.find('a', {"data-repository-hovercards-enabled":""}).text,
                t.find('a', {"data-hovercard-type":"repository"}).text
            )
            for t in soup.findAll("div", {"class": "Box-row"})
        ]
        
        for dependent in page_data:
            if dependent in dependents_data:
                list_end = True 
        
        if list_end:
            break
        else:    
            dependents_data = dependents_data + page_data
        
        try:
            paginationContainer = soup.find("div", {"class":"paginate-container"}).find_all('a')
        except:
            break
        
        try:
            if len(paginationContainer) > 1:
                paginationContainer = paginationContainer[1]
            else:
                paginationContainer = paginationContainer[0]
        except:
            break
        
        if paginationContainer:
            url = paginationContainer["href"]
        else:
            break
        
    return dependents_data

In [4]:
def countdown(t):
    while t:
        mins, secs = divmod(t, 60)
        timeformat = "{:02d}:{:02d}".format(mins, secs)
        print(timeformat, end="\r")
        time.sleep(1)
        t -= 1
    print("\n\n\n\n\n")

In [5]:
assert load_dotenv(), 'Environment variables could not be loaded'
g = Github(getenv("GITHUB"))

In [6]:
awesome_url = "https://github.com/protontypes/open-sustainable-technology"
awesome_path = urlparse(awesome_url).path.strip("/")
filename = "README.md"

awesome_repo = g.get_repo(awesome_path)
awesome_content_encoded = awesome_repo.get_contents(
    urllib.parse.quote(filename)
).content
awesome_content = base64.b64decode(awesome_content_encoded)

filehandle = open(".awesome.md", "w")
filehandle.write(awesome_content.decode("utf-8"))
filehandle.close()
repo_dict = awesome2py.AwesomeList(".awesome.md")

print(repo_dict)

Photovoltaics and Solar Energy
 - pvlib-python - A set of documented functions for simulating the performance of photovoltaic energy systems. [https://github.com/pvlib/pvlib-python] - pvfactors - Open source view-factor model for diffuse shading and bifacial PV modeling. [https://github.com/SunPower/pvfactors] - gsee - Global Solar Energy Estimator. [https://github.com/renewables-ninja/gsee] - PVMismatch - An explicit Python PV system IV & PV curve trace calculator which can also calculate mismatch. [https://github.com/SunPower/PVMismatch] - rdtools - An open source library to support reproducible technical analysis of time series data from photovoltaic energy systems. [https://github.com/NREL/rdtools] - Machine-Learning-for-Solar-Energy-Prediction - Predict the power production of a solar panel farm from weather measurements using machine learning. [https://github.com/ColasGael/Machine-Learning-for-Solar-Energy-Prediction] - elpv-dataset - A dataset of functional and defective solar c

 - Datasets APIs and open source projects related to Climate Change - A curated list of APIs, open data and ML/AI projects on climate change. [https://github.com/KKulma/climate-change-data] - Awesome Green Software - Research, tools, code, libraries and training to for building applications that emit less carbon into our atmosphere. [https://github.com/Green-Software-Foundation/awesome-green-software] - Awesome Sustainability Jobs - A curated list of companies in the sustainability sector that have jobs for devs. [https://github.com/pogopaule/awesome-sustainability-jobs] - Awesome Spectral Indices - A ready-to-use curated list of Spectral Indices for Remote Sensing applications. [https://github.com/davemlz/awesome-ee-spectral-indices] - Awesome Vegetation Index - List of reference, applications of common Vegetation Indices for Multi-spectral, hyper-spectral and UAV images. [https://github.com/px39n/Awesome-Vegetation-Index] - awesome-transit - Community list of transit APIs, apps, data

In [7]:
csv_github_projects = open("./csv/github_projects.csv", "w", newline="")
csv_github_fieldnames = [
    "project_name",
    "oneliner",
    "git_namespace",
    "git_url",
    "platform",
    "topics",
    "rubric",
    "last_update",
    "stargazers_count",
    "number_of_dependents",
    "stars_last_year",
    "project_active_checkbox",
    "dominating_language",
    "organization",
    "organization_user_name",
    "languages",
    "homepage",
    "created",
    "license",
    "total_commits_last_year",
    "total_number_of_commits",
    "last_issue_closed",
    "open_issues",
    "closed_pullrequests",
    "closed_issues",
    "open_pullrequests",
    "reviews_per_pr",
    "community_development_distribution_score",
    "last_released_date",
    "last_release_tag_name",
    "good_first_issue",
    "contributors",
    "accepts_donations",
    "donation_platforms",
    "code_of_conduct",
    "contribution_guide",
    "organization_avatar",
    "dependents_repos"

]
writer_github_projects = csv.DictWriter(
    csv_github_projects, fieldnames=csv_github_fieldnames
)
writer_github_projects.writeheader()


csv_github_organizations_fieldnames = [
    "organization_name",
    "organization_user_name",
    "organization_github_url",
    "organization_website",
    "organization_location",
    "organization_avatar",
    "organization_public_repos",
    "organization_created",
    "organization_last_update",
]

csv_github_organizations_read = open("./csv/github_organizations_back.csv", "r", newline="")
reader = csv.DictReader(csv_github_organizations_read)
for row in reader:
    print(row)

csv_github_organizations = open("./csv/github_organizations.csv", "w", newline="")


                            
writer_github_organizations = csv.DictWriter(
    csv_github_organizations, fieldnames=csv_github_organizations_fieldnames
)
writer_github_organizations.writeheader()
organization_list = []

csv_gitlab_projects = open("./csv/gitlab_projects.csv", "w", newline="")
csv_gitlab_fieldnames = [
    "project_name",
    "oneliner",
    "git_namespace",
    "git_url",
    "rubric",
]
writer_gitlab_projects = csv.DictWriter(
    csv_gitlab_projects, fieldnames=csv_gitlab_fieldnames
)
writer_gitlab_projects.writeheader()

csv_custom_platform_projects = open(
    "./csv/custom_platform_projects.csv", "w", newline=""
)
csv_custom_platform_fieldnames = [
    "project_name",
    "oneliner",
    "git_namespace",
    "git_url",
    "rubric",
]
writer_custom_platform_projects = csv.DictWriter(
    csv_custom_platform_projects, fieldnames=csv_custom_platform_fieldnames
)
writer_custom_platform_projects.writeheader()

{'organization_name': 'pvlib', 'organization_user_name': 'pvlib', 'organization_github_url': 'https://github.com/pvlib', 'organization_website': '', 'organization_location': '', 'organization_avatar': 'https://avatars.githubusercontent.com/u/11037261?v=4', 'organization_public_repos': '4', 'organization_created': '2015/02/17, 00:04:09', 'organization_last_update': '2021/11/22, 10:16:29'}
{'organization_name': 'SunPower', 'organization_user_name': 'SunPower', 'organization_github_url': 'https://github.com/SunPower', 'organization_website': '', 'organization_location': '', 'organization_avatar': 'https://avatars.githubusercontent.com/u/1341977?v=4', 'organization_public_repos': '18', 'organization_created': '2012/01/18, 01:25:32', 'organization_last_update': '2021/10/29, 19:53:01'}
{'organization_name': 'Renewables.ninja', 'organization_user_name': 'renewables-ninja', 'organization_github_url': 'https://github.com/renewables-ninja', 'organization_website': 'https://www.renewables.ninja/'

{'organization_name': 'rOpenSci', 'organization_user_name': 'ropensci', 'organization_github_url': 'https://github.com/ropensci', 'organization_website': 'https://ropensci.org/', 'organization_location': 'Berkeley, CA', 'organization_avatar': 'https://avatars.githubusercontent.com/u/1200269?v=4', 'organization_public_repos': '369', 'organization_created': '2011/11/16, 19:26:47', 'organization_last_update': '2021/11/26, 22:23:09'}
{'organization_name': 'GRECO project', 'organization_user_name': 'greco-project', 'organization_github_url': 'https://github.com/greco-project', 'organization_website': 'https://www.greco-project.eu/', 'organization_location': '', 'organization_avatar': 'https://avatars.githubusercontent.com/u/50671643?v=4', 'organization_public_repos': '7', 'organization_created': '2019/05/15, 15:39:28', 'organization_last_update': '2021/07/28, 07:50:42'}
{'organization_name': 'Materials Design Group', 'organization_user_name': 'WMD-group', 'organization_github_url': 'https:/

{'organization_name': '', 'organization_user_name': 'python-windrose', 'organization_github_url': 'https://github.com/python-windrose', 'organization_website': '', 'organization_location': '', 'organization_avatar': 'https://avatars.githubusercontent.com/u/28726174?v=4', 'organization_public_repos': '1', 'organization_created': '2017/05/16, 08:09:46', 'organization_last_update': '2021/11/24, 15:35:11'}
{'organization_name': 'Imperial College London', 'organization_user_name': 'ImperialCollegeLondon', 'organization_github_url': 'https://github.com/ImperialCollegeLondon', 'organization_website': '', 'organization_location': 'Imperial College London', 'organization_avatar': 'https://avatars.githubusercontent.com/u/1220306?v=4', 'organization_public_repos': '332', 'organization_created': '2011/11/25, 14:16:50', 'organization_last_update': '2021/11/26, 22:41:24'}
{'organization_name': 'brightwind', 'organization_user_name': 'brightwind-dev', 'organization_github_url': 'https://github.com/br

{'organization_name': 'Toyota Research Institute - Accelerated Materials Design & Discovery (AMDD)', 'organization_user_name': 'TRI-AMDD', 'organization_github_url': 'https://github.com/TRI-AMDD', 'organization_website': '', 'organization_location': '', 'organization_avatar': 'https://avatars.githubusercontent.com/u/62159728?v=4', 'organization_public_repos': '12', 'organization_created': '2020/03/13, 22:13:16', 'organization_last_update': '2021/11/19, 22:08:50'}
{'organization_name': 'QuESt', 'organization_user_name': 'snl-quest', 'organization_github_url': 'https://github.com/snl-quest', 'organization_website': 'https://www.sandia.gov/ess-ssl/tools/quest/', 'organization_location': '', 'organization_avatar': 'https://avatars.githubusercontent.com/u/50459090?v=4', 'organization_public_repos': '1', 'organization_created': '2019/05/09, 16:44:53', 'organization_last_update': '2021/11/26, 10:48:35'}
{'organization_name': '', 'organization_user_name': 'batterysim', 'organization_github_url

{'organization_name': 'OpenIDEAS', 'organization_user_name': 'open-ideas', 'organization_github_url': 'https://github.com/open-ideas', 'organization_website': '', 'organization_location': '', 'organization_avatar': 'https://avatars.githubusercontent.com/u/5467967?v=4', 'organization_public_repos': '5', 'organization_created': '2013/09/16, 08:15:06', 'organization_last_update': '2021/11/08, 11:47:01'}
{'organization_name': '', 'organization_user_name': 'HotMaps', 'organization_github_url': 'https://github.com/HotMaps', 'organization_website': 'https://www.hotmaps-project.eu/', 'organization_location': '', 'organization_avatar': 'https://avatars.githubusercontent.com/u/25738377?v=4', 'organization_public_repos': '44', 'organization_created': '2017/02/13, 08:38:39', 'organization_last_update': '2021/11/17, 06:19:04'}
{'organization_name': '', 'organization_user_name': 'EDF-TREE', 'organization_github_url': 'https://github.com/EDF-TREE', 'organization_website': '', 'organization_location':

...

{'organization_name': 'OpenFLUID', 'organization_user_name': 'OpenFLUID', 'organization_github_url': 'https://github.com/OpenFLUID', 'organization_website': 'http://www.openfluid-project.org', 'organization_location': 'Montpellier, France', 'organization_avatar': 'https://avatars.githubusercontent.com/u/3300491?v=4', 'organization_public_repos': '18', 'organization_created': '2013/01/17, 22:20:21', 'organization_last_update': '2021/11/18, 14:57:05'}
{'organization_name': '', 'organization_user_name': 'ConservationInternational', 'organization_github_url': 'https://github.com/ConservationInternational', 'organization_website': '', 'organization_location': '', 'organization_avatar': 'https://avatars.githubusercontent.com/u/6778872?v=4', 'organization_public_repos': '45', 'organization_created': '2014/02/25, 05:08:46', 'organization_last_update': '2021/11/19, 13:57:43'}
{'organization_name': 'COAL Capstone', 'organization_user_name': 'capstone-coal', 'organization_github_url': 'https://gi

{'organization_name': 'Sizespectrum', 'organization_user_name': 'sizespectrum', 'organization_github_url': 'https://github.com/sizespectrum', 'organization_website': '', 'organization_location': '', 'organization_avatar': 'https://avatars.githubusercontent.com/u/13071057?v=4', 'organization_public_repos': '10', 'organization_created': '2015/06/26, 22:39:44', 'organization_last_update': '2021/11/26, 16:42:38'}
{'organization_name': 'yunity', 'organization_user_name': 'yunity', 'organization_github_url': 'https://github.com/yunity', 'organization_website': 'https://yunity.org', 'organization_location': '', 'organization_avatar': 'https://avatars.githubusercontent.com/u/14811840?v=4', 'organization_public_repos': '23', 'organization_created': '2015/09/23, 21:20:46', 'organization_last_update': '2021/11/24, 03:43:26'}
{'organization_name': 'Open Food Foundation', 'organization_user_name': 'openfoodfoundation', 'organization_github_url': 'https://github.com/openfoodfoundation', 'organizatio

{'organization_name': 'The Environment Ontology', 'organization_user_name': 'EnvironmentOntology', 'organization_github_url': 'https://github.com/EnvironmentOntology', 'organization_website': 'http://www.environmentontology.org', 'organization_location': '', 'organization_avatar': 'https://avatars.githubusercontent.com/u/11706544?v=4', 'organization_public_repos': '11', 'organization_created': '2015/03/29, 13:45:39', 'organization_last_update': '2021/11/17, 19:46:33'}
{'organization_name': 'Open SDG', 'organization_user_name': 'open-sdg', 'organization_github_url': 'https://github.com/open-sdg', 'organization_website': '', 'organization_location': '', 'organization_avatar': 'https://avatars.githubusercontent.com/u/45497269?v=4', 'organization_public_repos': '12', 'organization_created': '2018/11/30, 19:13:01', 'organization_last_update': '2021/11/23, 15:06:56'}
{'organization_name': '', 'organization_user_name': 'SDG-InterfaceOntology', 'organization_github_url': 'https://github.com/SD

{'organization_name': 'Awesome Cryosphere', 'organization_user_name': 'awesome-cryosphere', 'organization_github_url': 'https://github.com/awesome-cryosphere', 'organization_website': '', 'organization_location': '', 'organization_avatar': 'https://avatars.githubusercontent.com/u/84785608?v=4', 'organization_public_repos': '1', 'organization_created': '2021/05/25, 13:27:18', 'organization_last_update': '2021/11/07, 19:17:40'}
{'organization_name': 'Software Underground', 'organization_user_name': 'softwareunderground', 'organization_github_url': 'https://github.com/softwareunderground', 'organization_website': 'https://softwareunderground.org', 'organization_location': '', 'organization_avatar': 'https://avatars.githubusercontent.com/u/8144157?v=4', 'organization_public_repos': '37', 'organization_created': '2014/07/12, 15:14:48', 'organization_last_update': '2021/11/26, 11:41:04'}
{'organization_name': "Arctic Basecamp's Arctic Risk", 'organization_user_name': 'arctic-risk', 'organiza

52

In [ ]:
for r in repo_dict.rubrics:
    for entry in r.entries:
        print("------------------------")
        print("Processing %s" % entry.name)
        print("URL: %s" % entry.url)
        if urlparse(entry.url).netloc == "github.com":
            print("%s is a GitHub project" % entry.name)
            try:

                remaining, limit = g.rate_limiting
                resettime = g.rate_limiting_resettime
                min_requests = 300
                if remaining < min_requests:
                    print("------------------------")
                    print("Waiting for more available GitHub requests:")
                    current_time = datetime.now().timestamp()
                    countdown((int(resettime) - int(current_time) + 2))

                print(
                    "GitHub Requests | Limit: {}, Remaining: {}".format(
                        limit, remaining
                    )
                )
                # Gather project information from GitHub
                # https://pygithub.readthedocs.io/en/latest/github_objects/Repository.html
                repo_path = urlparse(entry.url).path.strip("/")
                platform = "github"
                user, project_name = path.split(repo_path)
                repo = g.get_repo(repo_path)
                contents_root = repo.get_contents("")
                releases = repo.get_releases()
                commits = repo.get_commits()
                stargazers = repo.get_stargazers_with_dates()
                
                # Crawel dependents
                try:
                    dependents_repos = crawl_dependents(repo_path,20)
                    
                except Exception as e:
                    print("Dependents not available:")
                    print(e)
                    dependents_repo = []
                    
                    
                number_of_dependents = len(dependents_repos)
                dependents_repos = ",".join(dependents_repos)
                
                # Check if the project is still active
                closed_issues = repo.get_issues(state="closed")
                open_issues = repo.get_issues(state="open")

                closed_pullrequests = repo.get_pulls(state="closed")
                open_pullrequests = repo.get_pulls(state="open")
                
                closed_prs = closed_pullrequests.totalCount
                open_prs = open_pullrequests.totalCount
                
                if closed_prs > 10:
                    pr_review_analyse = 10
                else:
                    pr_review_analyse = closed_prs
                
                total_reviews = 0
                for pull_request in closed_pullrequests[0:pr_review_analyse-1]:
                    pr_reviews = pull_request.get_reviews()
                    total_reviews = total_reviews + pr_reviews.totalCount
                
                try:
                    reviews_per_pr = total_reviews/pr_review_analyse
                    
                except:                   
                    reviews_per_pr = 0

                if closed_issues.totalCount > 0:
                    last_issue_closed = closed_issues[0].updated_at

                total_commits_last_year = 0
                activity = repo.get_stats_commit_activity()
                for event in activity:
                    total_commits_last_year = total_commits_last_year + event.total

                last_issue_closed_timedelta = datetime.utcnow() - last_issue_closed

                if (
                    last_issue_closed_timedelta > timedelta(days=365)
                    and total_commits_last_year == 0
                    or repo.archived
                ):
                    print("%s is an inactive project" % entry.name)
                    project_active_checkbox = "false"
                else:
                    print("%s is an active project" % entry.name)
                    project_active_checkbox = "true"

                try:
                    license = repo.get_license()
                    if license.license.spdx_id == "NOASSERTION":
                        print("Custom license found")
                        license_name = "CUSTOM"
                    else:
                        license_name = license.license.spdx_id
                except:
                    print("No license information found")
                    license_name == "UNDEFINED"

                labels = ",".join([entry.name for entry in repo.get_labels()])
                topics = ",".join(repo.get_topics())

                languages_states = repo.get_languages()
                programming_languages = ",".join(languages_states.keys())
                
                try:
                    dominating_language = list(languages_states.keys())[0]
                except:
                    dominating_language = ""

                try:
                    dotfolder_file = repo.get_contents(".github")
                    for file in dotfolder_file:
                        if file.path == ".github/FUNDING.yml":
                            print("Funding found")
                            donation_platforms = base64.b64decode(file.content)
                            accepts_donations = "true"
                        else:
                            donation_platforms = None
                            accepts_donations = "false"
                except:
                    print("No funding information found")
                    donation_platforms = None
                    accepts_donations = "false"

                try:
                    code_of_conduct = "false"
                    contribution_guide = "false"
                    for file_content in contents_root:
                        if file_content.path.lower().find("code_of_conduct") != -1:
                            print("Code of conduct found")
                            code_of_conduct = "true"
                        if file_content.path.lower().find("contributing") != -1:
                            print("Contribution guide found")
                            contribution_guide = "true"

                    for file_content in dotfolder_file:
                        if file_content.path.lower().find("code_of_conduct") != -1:
                            print("Code of conduct found")
                            code_of_conduct = "true"
                        if file_content.path.lower().find("contributing") != -1:
                            print("Contribution guide found")
                            contribution_guide = "true"

                except Exception as e:
                    print(e)

                contributors = repo.get_stats_contributors()
                contributor_activity = {}
                commits_total = 0
                for individuum in contributors:
                    contributor_activity[individuum.author.login] = individuum.total
                    commits_total = commits_total + individuum.total

                sorted_contributor = dict(
                    sorted(contributor_activity.items(), key=lambda item: item[1])
                )
                weighted_contribution = {
                    k: v / commits_total for k, v in sorted_contributor.items()
                }

                # Create a very simple community health score that shows how much the project is focused on one developer
                community_development_distribution_score = 1 - max(
                    weighted_contribution.values()
                )

                try:
                    last_released_date = releases[0].published_at.strftime(
                        "%Y/%m/%d, %H:%M:%S"
                    )
                    last_release_tag_name = releases[0].tag_name

                except Exception as e:
                    print("No Release found")
                    last_released_date = ""
                    last_release_tag_name = ""
                    print(e)

                total_number_of_commits = commits.totalCount
                
                stars_last_year = 0
                for star in stargazers:
                    starred_delta = datetime.utcnow() - star.starred_at
                    if  starred_delta < timedelta(days=365):
                        stars_last_year = stars_last_year + 1

                # Gathering organization data
                if repo.organization is None:
                    print("No Organization found.Project in user namespace.")
                    organization_user_name = None

                elif repo.organization.login not in organization_list:
                    print("Organization not in list. Gathering data.")
                    organization_user_name = repo.organization.login
                    organization_name = repo.organization.name
                    organization_avatar = repo.organization.avatar_url
                    organization_public_repos = repo.organization.public_repos
                    organization_location = repo.organization.location
                    organization_github_url = repo.organization.html_url
                    organization_website = repo.organization.blog
                    organization_created = repo.organization.created_at.strftime("%Y/%m/%d, %H:%M:%S")
                    organization_repos = g.search_repositories(query='user:'+organization_user_name, sort='updated')     
                    organization_last_update = organization_repos[0].updated_at.strftime("%Y/%m/%d, %H:%M:%S")
                        
                        
                    organization_data = {
                        "organization_name": organization_name,
                        "organization_user_name":organization_user_name,
                        "organization_github_url":organization_github_url,
                        "organization_website":organization_website,
                        "organization_avatar": organization_avatar,
                        "organization_location": organization_location,
                        "organization_public_repos": organization_public_repos,
                        "organization_created": organization_created,
                        "organization_last_update": organization_last_update
                    }

                    organization_list.append(organization_user_name)
                    print("Organization Data:")
                    print(organization_data)
                    writer_github_organizations.writerow(organization_data)

                else:
                    organization_name = repo.organization.login
                    organization_user_name = repo.organization.name
                    organization_avatar = repo.organization.avatar_url
                    

                project_data = {
                    "project_name": entry.name,
                    "git_namespace": user,
                    "git_url": repo.clone_url,
                    "rubric": r.key,
                    "oneliner": entry.text[2:],
                    "topics": topics,
                    "organization": organization_name,
                    "organization_user_name": organization_user_name,
                    "created": repo.created_at.strftime("%Y/%m/%d, %H:%M:%S"),
                    "last_update": repo.updated_at.strftime("%Y/%m/%d, %H:%M:%S"),
                    "total_commits_last_year": total_commits_last_year,
                    "project_active_checkbox": project_active_checkbox,
                    "last_issue_closed": last_issue_closed.strftime(
                        "%Y/%m/%d, %H:%M:%S"
                    ),
                    "last_released_date": last_released_date,
                    "last_release_tag_name": last_release_tag_name,
                    "total_number_of_commits": total_number_of_commits,
                    "community_development_distribution_score": community_development_distribution_score,
                    "stargazers_count": repo.stargazers_count,
                    "number_of_dependents": number_of_dependents,
                    "dependents_repos": dependents_repos,
                    "stars_last_year": stars_last_year,
                    "dominating_language": dominating_language,
                    "languages": programming_languages,
                    "homepage": repo.homepage,
                    "closed_issues": closed_issues.totalCount,
                    "open_issues": open_issues.totalCount,
                    "closed_pullrequests": closed_prs,
                    "open_pullrequests": open_prs,
                    "reviews_per_pr": reviews_per_pr,
                    "good_first_issue": repo.get_issues(
                        state="open", labels=["good first issue"]
                    ).totalCount,
                    "license": license_name,
                    "contributors": repo.get_contributors().totalCount,
                    "accepts_donations": accepts_donations,
                    "donation_platforms": donation_platforms,
                    "code_of_conduct": code_of_conduct,
                    "contribution_guide": contribution_guide,
                    "organization_avatar":organization_avatar,
                    "platform":platform
                }

                print("Project Data:")
                print(project_data)
                writer_github_projects.writerow(project_data)

            except Exception as e:
                print(colored("Failed to gather project information:"))
                print(colored(e, "red"))
                print(traceback.format_exc())

        elif urlparse(entry.url).netloc == "gitlab.com":
            print("%s is a Gitlab project" % entry.name)
            repo_path = urlparse(entry.url).path.strip("/")
            user, project_name = path.split(repo_path)
            platform = "gitlab"

            project_data = {
                "project_name": entry.name,
                "git_namespace": user,
                "git_url": entry.url,
                "rubric": r.key,
                "oneliner": entry.text[2:],
                "platform":platform
            }
            print("Project Data:")
            print(project_data)
            writer_github_projects.writerow(project_data)

        else:
            print("%s is hosted on custom platform" % entry.name)
            repo_path = urlparse(entry.url).path.strip("/")
            user, project_name = path.split(repo_path)
            platform = "custom"

            project_data = {
                "project_name": entry.name,
                "git_namespace": user,
                "homepage": entry.url,
                "rubric": r.key,
                "oneliner": entry.text[2:],
                "platform":platform
            }

            print("Project Data:")
            print(project_data)
            writer_github_projects.writerow(project_data)

csv_github_projects.close()
csv_github_organizations.close()
csv_gitlab_projects.close()
csv_custom_platform_projects.close()

------------------------
Processing pvlib-python
URL: https://github.com/pvlib/pvlib-python
pvlib-python is a GitHub project
GitHub Requests | Limit: 5000, Remaining: 4403
pvlib-python is an active project
Code of conduct found
Contribution guide found
Organization not in list. Gathering data.
Organization Data:
{'organization_name': 'pvlib', 'organization_user_name': 'pvlib', 'organization_github_url': 'https://github.com/pvlib', 'organization_website': '', 'organization_avatar': 'https://avatars.githubusercontent.com/u/11037261?v=4', 'organization_location': None, 'organization_public_repos': 4, 'organization_created': '2015/02/17, 00:04:09', 'organization_last_update': '2022/02/13, 15:50:13'}
Project Data:
{'project_name': 'pvlib-python', 'git_namespace': 'pvlib', 'git_url': 'https://github.com/pvlib/pvlib-python.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'A set of documented functions for simulating the performance of photovoltaic energy systems.', 'topics': 'sol

{'project_name': 'pvfactors', 'git_namespace': 'SunPower', 'git_url': 'https://github.com/SunPower/pvfactors.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Open source view-factor model for diffuse shading and bifacial PV modeling.', 'topics': 'solar-energy,renewable-energy,python,bifacial', 'organization': 'SunPower', 'organization_user_name': 'SunPower', 'created': '2018/05/14, 06:10:55', 'last_update': '2022/01/13, 00:44:36', 'total_commits_last_year': 8, 'project_active_checkbox': 'true', 'last_issue_closed': '2022/01/20, 23:49:19', 'last_released_date': '2021/03/27, 13:39:44', 'last_release_tag_name': 'v1.5.1', 'total_number_of_commits': 102, 'community_development_distribution_score': 0.10989010989010994, 'stargazers_count': 55, 'number_of_dependents': 5, 'dependents_repos': 'thesethtruth/LESO,prathamrg/solarapp,anomam/pvfactors_iea_pvps_study,toddkarin/vocmax,toddkarin/pvtools', 'stars_last_year': 14, 'dominating_language': 'Python', 'languages': 'Python,Makefile

{'project_name': 'Machine-Learning-for-Solar-Energy-Prediction', 'git_namespace': 'ColasGael', 'git_url': 'https://github.com/ColasGael/Machine-Learning-for-Solar-Energy-Prediction.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Predict the power production of a solar panel farm from weather measurements using machine learning.', 'topics': 'machine-learning,neural-network,data-processing,python,matlab,tensorflow', 'organization': 'National Renewable Energy Laboratory', 'organization_user_name': None, 'created': '2018/05/06, 19:43:04', 'last_update': '2022/02/10, 17:21:34', 'total_commits_last_year': 0, 'project_active_checkbox': 'false', 'last_issue_closed': '2019/11/15, 19:01:02', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 11, 'community_development_distribution_score': 0.0, 'stargazers_count': 120, 'number_of_dependents': 0, 'dependents_repos': '', 'stars_last_year': 34, 'dominating_language': 'Python', 'languages': 'Python,MATLAB

{'project_name': 'pvcaptest', 'git_namespace': 'pvcaptest', 'git_url': 'https://github.com/pvcaptest/pvcaptest.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Collection of functions and Jupyter Notebooks to partially automate running a capacity test following ASTM E2848.', 'topics': '', 'organization': None, 'organization_user_name': 'pvcaptest', 'created': '2017/09/19, 01:33:23', 'last_update': '2021/12/02, 22:14:36', 'total_commits_last_year': 36, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/07/25, 16:51:03', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 606, 'community_development_distribution_score': 0.0018832391713747842, 'stargazers_count': 13, 'number_of_dependents': 1, 'dependents_repos': 'pvcaptest/pvcaptest', 'stars_last_year': 3, 'dominating_language': 'Python', 'languages': 'Python', 'homepage': None, 'closed_issues': 39, 'open_issues': 28, 'closed_pullrequests': 15, 'open_pullrequests': 2, 'reviews_per_pr

{'project_name': 'SolarPV-DER-simulation-utility', 'git_namespace': 'tdcosim', 'git_url': 'https://github.com/tdcosim/SolarPV-DER-simulation-tool.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Allows user to run dynamics simulations for solar photovoltaic distributed energy resource connected to a stiff voltage source or to an external program.', 'topics': '', 'organization': 'TDcoSim Team', 'organization_user_name': 'tdcosim', 'created': '2019/03/19, 21:59:40', 'last_update': '2022/01/26, 05:14:57', 'total_commits_last_year': 35, 'project_active_checkbox': 'true', 'last_issue_closed': '2021/09/09, 05:07:30', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 205, 'community_development_distribution_score': 0.014925373134328401, 'stargazers_count': 21, 'number_of_dependents': 1, 'dependents_repos': 'tdcosim/TDcoSim', 'stars_last_year': 5, 'dominating_language': 'Python', 'languages': 'Python', 'homepage': '', 'closed_issues': 26, 'open_iss

...

{'project_name': 'The Hydrogen Risk Assessment Models', 'git_namespace': 'sandialabs', 'git_url': 'https://github.com/sandialabs/hyram.git', 'rubric': 'Hydrogen', 'oneliner': 'The first-ever software toolkit that integrates deterministic and probabilistic models for quantifying accident scenarios, predicting physical effects, and characterizing hydrogen hazards impact on people and structures.', 'topics': 'snl-applications', 'organization': 'Sandia National Laboratories', 'organization_user_name': 'sandialabs', 'created': '2019/05/07, 16:06:52', 'last_update': '2022/02/12, 05:22:28', 'total_commits_last_year': 2, 'project_active_checkbox': 'true', 'last_issue_closed': '2022/01/13, 03:46:54', 'last_released_date': '', 'last_release_tag_name': '', 'total_number_of_commits': 6, 'community_development_distribution_score': 0.16666666666666663, 'stargazers_count': 22, 'number_of_dependents': 0, 'dependents_repos': '', 'stars_last_year': 6, 'dominating_language': 'C#', 'languages': 'C#,Python

URL: https://github.com/NREL/hescore-hpxml
HPXML to Home Energy Score Translator is a GitHub project
GitHub Requests | Limit: 5000, Remaining: 775
HPXML to Home Energy Score Translator is an active project
Project Data:
{'project_name': 'HPXML to Home Energy Score Translator', 'git_namespace': 'NREL', 'git_url': 'https://github.com/NREL/hescore-hpxml.git', 'rubric': 'Buildings and Cities', 'oneliner': 'This translator script takes an HPXML file or directory of files as an input and generates HEScore inputs from it.', 'topics': '', 'organization': 'NREL', 'organization_user_name': 'National Renewable Energy Laboratory', 'created': '2014/09/18, 16:59:56', 'last_update': '2022/01/04, 20:07:24', 'total_commits_last_year': 194, 'project_active_checkbox': 'true', 'last_issue_closed': '2022/01/13, 15:45:40', 'last_released_date': '2021/07/02, 20:04:39', 'last_release_tag_name': 'v6.2', 'total_number_of_commits': 683, 'community_development_distribution_score': 0.4270650263620387, 'stargazers_

{'project_name': 'EnergyPlus', 'git_namespace': 'NREL', 'git_url': 'https://github.com/NREL/EnergyPlus.git', 'rubric': 'Buildings and Cities', 'oneliner': 'A whole building energy simulation program that engineers, architects, and researchers use to model both energy consumption and water usage in buildings.', 'topics': '', 'organization': 'NREL', 'organization_user_name': 'National Renewable Energy Laboratory', 'created': '2013/11/22, 14:47:34', 'last_update': '2022/02/13, 00:50:45', 'total_commits_last_year': 3785, 'project_active_checkbox': 'true', 'last_issue_closed': '2022/02/11, 17:04:20', 'last_released_date': '2022/02/07, 21:36:04', 'last_release_tag_name': 'v9.6-py3.7', 'total_number_of_commits': 28184, 'community_development_distribution_score': 0.8446756090884204, 'stargazers_count': 698, 'number_of_dependents': 0, 'dependents_repos': '', 'stars_last_year': 141, 'dominating_language': 'C++', 'languages': 'C++,Fortran,Python,VBA,CMake,C,REALbasic,Batchfile,Shell,Qt Script,HTM

{'organization_name': 'Ecotope, Inc.', 'organization_user_name': 'EcotopeResearch', 'organization_github_url': 'https://github.com/EcotopeResearch', 'organization_website': 'www.ecotope.com', 'organization_avatar': 'https://avatars.githubusercontent.com/u/10763891?v=4', 'organization_location': 'Seattle, WA', 'organization_public_repos': 14, 'organization_created': '2015/01/29, 21:09:13', 'organization_last_update': '2021/12/15, 00:15:37'}
Project Data:
{'project_name': 'HPWHsim', 'git_namespace': 'EcotopeResearch', 'git_url': 'https://github.com/EcotopeResearch/HPWHsim.git', 'rubric': 'Buildings and Cities', 'oneliner': 'An open source simulation model for Heat Pump Water Heaters (HPWH).', 'topics': '', 'organization': 'Ecotope, Inc.', 'organization_user_name': 'EcotopeResearch', 'created': '2015/12/04, 01:24:35', 'last_update': '2021/12/15, 00:15:37', 'total_commits_last_year': 82, 'project_active_checkbox': 'true', 'last_issue_closed': '2022/02/11, 21:22:18', 'last_released_date': '

{'project_name': 'TEASER', 'git_namespace': 'RWTH-EBC', 'git_url': 'https://github.com/RWTH-EBC/TEASER.git', 'rubric': 'Buildings and Cities', 'oneliner': 'Tool for Energy Analysis and Simulation for Efficient Retrofit.', 'topics': 'python,simulation,buildings,urban-energy-modeling,hacktoberfest', 'organization': 'RWTH-EBC', 'organization_user_name': 'RWTH Aachen University - E.ON Energy Research Center - Institute for Energy Efficient Buildings and Indoor Climate', 'created': '2015/10/22, 13:57:56', 'last_update': '2021/12/15, 21:29:41', 'total_commits_last_year': 6, 'project_active_checkbox': 'true', 'last_issue_closed': '2022/01/18, 13:32:46', 'last_released_date': '2021/07/23, 13:12:18', 'last_release_tag_name': 'v0.7.6', 'total_number_of_commits': 2574, 'community_development_distribution_score': 0.574385510996119, 'stargazers_count': 67, 'number_of_dependents': 4, 'dependents_repos': 'illacceptanything/illacceptanything,Helarga/geojson-modelica-translator,urbanopt/geojson-modelic